In [1]:
import numpy as np

In [6]:
a = np.random.rand(5)
a

array([0.72038275, 0.76286824, 0.82898522, 0.32229129, 0.90576521])

In [9]:
indexes = np.array([0, 1])
~indexes

array([-1, -2])

In [5]:
a

array([0.28177852, 0.86821325, 0.97790898, 0.64812911, 0.86352816])

In [69]:
@numba.jit(nopython=True)
def greedy_mutual_information(forbidden, size, tot):
    return np.random.choice(np.delete(np.arange(tot), forbidden), size=size, replace=False)

In [71]:
greedy_mutual_information(np.empty(0, dtype=np.int64), 3, 5)

array([3, 4, 2])

In [97]:
import numba

@numba.jit(nopython=True)
def insert_numba(forbidden, current_elt):
    for elt in forbidden:
        if current_elt == elt:
            return False
    return True

@numba.jit(nopython=True)
def numba_top(input_arr, forbidden, size):
    count = 0
    output_top = np.zeros(size, dtype=np.int64)
    for ix in range(input_arr.shape[0]):
        current_elt = input_arr[ix]
        insert = insert_numba(forbidden, current_elt)
        if insert:
            output_top[count] = current_elt
            count += 1
        if count >= size:
            break
    return output_top

In [101]:
a = np.arange(1, 10, dtype=int)[::-1]
b = np.array([4, 8], dtype=int)

numba_top(a, b, 7)

array([9, 7, 6, 5, 3, 2, 1])

In [92]:
@numba.jit(nopython=True)
def numba_top(input_arr, forbidden, size):
    count = 0
    output_top = np.zeros(size, dtype=np.int64)
    for ix in range(input_arr.shape[0]):
        current_elt = input_arr[ix]
        insert = True
        for elt in forbidden:
            if current_elt == elt:
                insert = False
        if insert:
            output_top[count] = current_elt
            count += 1
        if count >= size:
            break
    return output_top

@numba.jit(nopython=True)
def skel(sampled_preferences, n_slots):
    expected_preferences = np.sum(sampled_preferences, 0) / sampled_preferences.shape[0]
    
    item_indexes_decreasing_expected_preferences = np.argsort(-expected_preferences)

    all_actions = np.ones((2 * n_slots, n_slots), dtype=np.int64)
    for k in range(1, n_slots + 1):
        items = item_indexes_decreasing_expected_preferences[:k]
        all_actions[k-1, :k] = items
        other_items = greedy_mutual_information(items, n_slots - k, sampled_preferences.shape[1])

        all_actions[k-1, k:] = other_items

    for k in range(1, n_slots + 1):
        items = greedy_mutual_information(np.empty(shape=0, dtype=np.int64), k, sampled_preferences.shape[1])
        all_actions[n_slots - 1 + k, :k] = items
        other_items = numba_top(item_indexes_decreasing_expected_preferences, items, n_slots - k)
        all_actions[n_slots - 1 + k, k:] = other_items
    return all_actions

In [93]:
sampled_preferences = np.random.rand(5).reshape(1, -1)
sampled_preferences

array([[0.87555112, 0.44915901, 0.51874898, 0.34566077, 0.42106011]])

In [94]:
skel(sampled_preferences, 3)

array([[0, 3, 1],
       [0, 2, 4],
       [0, 2, 1],
       [4, 0, 2],
       [2, 3, 0],
       [1, 3, 4]])

In [1]:
from ids_agents import EpochSamplingIDS

In [2]:
N = 5
K = 3

In [3]:
agent = EpochSamplingIDS(action_type="approximate",
                         k=K,
                         n=N,
                         correlated_sampling=False,
                         limited_prefs=False,
                         n_samples=10,
                         info_type="gain")

Agent believes first=best? False
Info type is: gain
Action selection+approximate
scaling factor: 0.0


In [4]:
agent.sample_from_posterior(1)

array([[28.63552059,  5.24950891,  8.02682574,  0.08707483,  3.78314976]])

In [14]:
agent.proposal()

array([3, 2, 1])